<a href="https://colab.research.google.com/github/dyjdlopez/fund_opts_python/blob/main/metaheuristics/metaopts_01_01.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Lesson 1.1: Structures and Functions for Computational Optimization
$_\text{Metahuristics and Optimization | D.J.D. Lopez | 2026}$


Computational optimization is fundamentally about evaluating and comparing many candidate solutions in order to find one that minimizes (or maximizes) a given objective, subject to constraints. To do this effectively in Python, it is important to structure code so that mathematical objects such as objective functions, variable vectors, and iterative algorithms are represented cleanly and consistently.


In this lesson, the focus is on three core ingredients:

1. **Functions**, which encode objectives and constraints as callable pieces of code.

2. **Collections**, especially lists, which store sets of decision variables, parameters, and time series data.

3. **Iterators and iterative routines**, which systematically traverse these collections to implement search and improvement algorithms.

Together, these ideas bridge the gap between mathematical optimization formulations and executable Python code that can evaluate, compare, and update candidate solutions.



## 1. Functions
In optimization, a **mathematical function** is a mapping from a set of inputs (often vectors of decision variables) to a single numerical output, such as cost, loss, or profit. For example, an objective function $f(x)$ might map a vector $x$ of decision variables to a real number measuring total cost, and optimization algorithms work by evaluating this function at many different points to find a minimum or maximum.

A **Python function** is a named block of code that takes inputs (arguments), executes a sequence of statements, and optionally returns a result using `return`. While a mathematical function is an abstract mapping, a Python function is an implementation that can include control flow, side effects (like printing or modifying data), and may or may not return a value, so a single Python function can encapsulate not only $f(x)$ but also logging, checks, and pre/post-processing.

In computational optimization, the typical pattern is: the mathematical object
$f(x)$ becomes a Python function `def f(x): ... return value`, which can then be passed into solvers or called in custom search loops to evaluate the objective for different candidate solutions.



### 1.1 Structure of a Python function

A basic Python function has three key structural components: a name, parameters, and an optional return value. The general syntax is:

    def function_name(parameters):
       # function body
    return result

* The **function name** identifies the function and is used when calling it.
* The **parameters** are placeholders for the input values the function operates on.
* the **return statement** sends a value back to the caller; if omitted, Python returns None by default

For example, we want to encode a function:
$$f(x)= ax^2 + bx + c$$

Although, $a$, $b$, and $c$ are constants while $x$ is are variables in the mathematical sense. However, in programming we can treat them all as variables as they can be changed depending on the nature of use of the function. We may opt to fix the values of $a$, $b$, and $c$ at a later time in development. For now we can translate the function as:

In [ ]:
def f(x, a, b, c):
  value = a*x**2 + b*x + c
  return value

We can now readily use the function for computation. Suppose $a = 1$, $b=8$, $c=15$, and $x=1$. We can then evaluate function:

In [ ]:
f(x=1, a=1, b=8, c=15)

24

However, when creating larger systems it might be confusing or hard to remember each function name. In software engineering, it is a considered best practice to name these functions according to their intended purpose in a given context. Let's say the original function $f(x)$ is actually used a cost function for a problem. We can rename it as `cost`:

In [ ]:
def cost(x, a, b, c):
  value = a*x**2 + b*x + c
  return value

### 1.2. Parameters

Parameters in a function definition specify what data the function expects to receive. When the function is called, **arguments** are passed to these parameters, and the body of the function uses them to perform its computation, such as evaluating an objective or constraint at a particular point.

Python supports several kinds of parameters:

* Positional parameters: `def f(x, y): ...`
* Default parameters: `def penalty(x, alpha=1.0): ...`

In optimization routines, parameters often represent:

* Decision variables (e.g., x as a scalar or list/vector).
* Model parameters (e.g., cost coefficients, step sizes, or penalty weights).
* Algorithmic settings (e.g., maximum iterations, tolerance).

Let's try to modify `cost` once more. Say we wish to set the default values of the coefficients for the functions so that we don't have to pass them as arguments everytime we would use the function. If we wish that $a$, $b$, and $c$ would have default values, but still have flexibility to change them later on we can program `cost` as:

In [ ]:
def cost(x, a=1, b=8, c=15):
  value = a*x**2 + b*x + c
  return value

In [ ]:
cost(1)

24

### 1.3. Return values vs “void” functions
In Python, a function can return a value or perform actions without returning anything useful (often called “void” by analogy, though the technical return value is `None`)

**Functions with return values**

These are central in optimization because algorithms repeatedly call such functions to get objective or constraint values.

In [ ]:
def objective(x):
    return (x - 3)**2 + 1

objective(3.14)

1.0196

Python also allows returning multiple values (typically packed in a tuple), which is convenient for returning both a solution and associated statistics from an optimization routine.

In [ ]:
def f_f_prime(x, a=1, b=8, c=15):
  f_val = f(x, a, b, c)
  f_prime_val = 2*a*x + b
  return f_val, f_prime_val

f_f_prime(2)

(35, 12)

**Void-like functions** (returning `None`)

These functions primarily produce side effects, such as printing intermediate results or logging iterations.

In [ ]:
def log_iteration(iter_num, x, f_val):
  print("="*50)
  print(f"Iter {iter_num:03}\t|  x = {x} |\tf(x) = {f_val:.3f}")

log_iteration(99, 20, 1.2571155)
log_iteration(100, 24, 0.051268)

Iter 099	|  x = 20 |	f(x) = 1.257
Iter 100	|  x = 24 |	f(x) = 0.051


Such functions are useful for monitoring optimization routines but are not themselves part of the mathematical model, since they do not return a numerical value to be minimized or maximized.

### 1.4 Specifying datatypes (type hints) for functions

Python is dynamically typed, so functions run without explicit type declarations, but type hints allow you to annotate parameter and return types to improve readability, static checking, and maintainability. Type hints use the syntax:

In [ ]:
def gradient_step(x: float, grad: float, step_size: float) -> float:
    return x - step_size * grad

In [ ]:
gradient_step(3.14, 1.2e-1, 1e-3)

3.1398800000000002

In [ ]:
gradient_step(1, 1.2e-1, 1e-3)

0.99988

Here, the annotations indicate that `x`, `grad`, and `step_size` are expected to be floats and that the function returns a `float`, which aligns well with typical numerical optimization routines.

*NOTE*: Python's type-hinting rules for numeric types deliberately relax strictness: a parameter annotated as `float` is allowed to receive an int, and a parameter annotated as `complex` is allowed to receive `float` or `int`. Concretely, PEP 484 states that when an argument is annotated as `float`, passing an `int` is acceptable, and when annotated as `complex`, passing `float` or int is acceptable, which mirrors the mathematical idea that any integer can be interpreted as a real, and any real as a complex number with zero imaginary part.



For collections of variables, you can use types from the `typing` module:

In [ ]:
from typing import List

def total_cost(xs: List[float]) -> float:
    return sum(xs)

### **Activity 1**: Defining functions


A. Translate the following equations as Python functions:
1. $\phi(\theta) = \cos(\theta) + i\sin(\theta)$
2. $s(f,t) = e^{-i2\pi ft}$
3. $h(x) =
\begin{cases}
  \beta x^2      & \text{if } -1 \geq x > 1, \\
  \gamma |x|       & \text{if otherwise}
\end{cases}$

Note that $i$ is an imaginary number $i=\sqrt{-1}$.

B. Translate the following Python functions into mathematical expressions. Use LaTeX in Jupyter Notebook:

In [ ]:
### Activity 2.1
import math
def area_circle(radius):
  return math.pi * radius**2

In [ ]:
### Activity 1.2
def weight_update(w, b, delta_E, delta_W, delta_b, eta=1e-3):

  gradient_W = delta_E / delta_W
  gradient_b = delta_E / delta_b

  w_star -= eta*gradient_W*w
  b_star -= eta*gradient_b*b

  return w_star, b_star

In [ ]:
### Activity 1.3
def switch(x, mu=1e-6):
  value = None
  if x % 2 == 0:
    value = 2**x / (x**2 + mu)
  else:
    value = math.exp(x) / (10**x + mu)
  return value

## 2. Collections

In Python, collections are objects that group multiple values into a single structure so they can be stored, passed to functions, and processed together. In computational optimization, collections are used to represent vectors of decision variables, sequences of time periods, sets of indices, and other structured data that appear in mathematical models.

The three basic collections in this lesson are **lists**, **tuples**, and **sets**. Each has distinct properties that align with different modeling needs: ordered versus unordered data, mutable versus immutable structures, and whether duplicates are allowed.


### 2.1 Lists: ordered, mutable sequences

A list is an ordered, mutable sequence of elements; items can be added, removed, or modified in place, and duplicates are allowed. Lists support indexing and slicing, which makes them useful for representing finite-dimensional decision vectors, time series data, or arrays of costs and constraints.

In [ ]:
hours = [0,1,2,3,4,5]

Elements in an ordered collection, like lists, can be accessed (retreived and modified) by their indeces. Note that in Python indexing starts at `0` rather than `1`.

In [ ]:
hours[0] #returns the zeroth hour from the hours list

0

Lists can also hold non-numerical values such as strings.

In [ ]:
cities = ['Barcelona', 'Los Angeles', 'Manila', 'Tokyo', 'Cape Town']

Lists can also be sliced using their indices. We can specify the start, end, and step. The general syntax for slicing a list is: `some_list[start:end:step]`.

In [ ]:
hours[1:5:2] ## this means get the hours starting from index 1, end before index 5, and take only every after 2 elements

[1, 3]

In [ ]:
hours[0:3:1] ## this means get the hours starting from index 0 and before index 3

[0, 1, 2]

In [ ]:
## In some cases if the slice starts with index 0 or that the step is 1, they can be omitted.
hours[:3]

[0, 1, 2]

In [ ]:
## Or if the ending index is the last index of the list
hours[1:]

[1, 2, 3, 4, 5]

In [ ]:
## A special technique in list slicing is that it can be used for reversing the order of a list
hours[::-1]

[5, 4, 3, 2, 1, 0]

### 2.2 Tuples: ordered, immutable records
A tuple is also an ordered sequence, but it is immutable: once created, its elements cannot be changed, although duplicates and indexing are still allowed. Tuples are well-suited to represent fixed records or points that should not be modified accidentally, such as a particular candidate solution or a fixed set of parameters.


In [ ]:
coords = (2,3)

Tuples can be aggregated as lists in some data sources like geospatial data.

In [ ]:
locations = [
    (41.38879, 2.15899),
    (34.0549, 118.2426),
    (14.5995, 120.9842),
    (35.6764, 139.6500),
    (-33.92, 18.42)]

In [ ]:
index = 0
print(f"Coordinates of {cities[index]} is {locations[index]}.")

Coordinates of Barcelona is (41.38879, 2.15899).


### 2.3 Sets: unordered collections of unique elements
A set is an unordered collection of unique, hashable elements; duplicates are automatically removed, and indexing is not supported. Sets support fast membership tests and set-theoretic operations such as union, intersection, and difference, which align well with how index sets and feasible regions are described in many optimization problems.


In [ ]:
blood_types = {'A', 'B', 'AB', 'O'}

Conversion between sets and lists is possible. This can be invoked by passing the collection into the instantiation object respective of the data type.

In [ ]:
### Converting a List to a Set (deletes duplicates)
attendees = ['202011223','202014100','202190331','202014660','4138','202014660','202014100', '2022119903', '202011223']
unique_attend = set(attendees)

print(f'Total number of instances: {len(attendees)}')
print(f'Unique Attendee count: {len(unique_attend)}')
print(unique_attend)

Total number of instances: 9
Unique Attendee count: 6
{'2022119903', '202011223', '202190331', '4138', '202014660', '202014100'}


### 2.4 Vectors
In mathematical optimization, a **vector** $x = (x_1, \dots, x_n)$ is a finite ordered list of numbers representing a point in $\mathbb{R}^n$ or $\mathbb{Z}^n$. In Python, this kind of finite-dimensional vector is most naturally represented using ordered collections: lists or tuples.  

- A **list** can model a mutable decision vector, for example $x = [x_0, x_1, x_2]$, which an algorithm updates in place during an iterative routine.  
- A **tuple** can model an immutable vector, for example $(x_0, x_1, x_2)$ stored as a fixed candidate solution or used as a key in dictionaries or elements of sets to record visited states.  
- A **set** can represent a collection of indices or vectors without duplicates, e.g. a set of feasible integer points or a set of active constraints, where order does not matter but uniqueness and membership tests are important.  

In more advanced numerical work, these Python collections are often converted to NumPy arrays to obtain true numerical vectors and benefit from vectorized operations and optimized linear algebra. However, conceptually in this lesson, a “vector of decision variables” can be read directly as a Python list or tuple whose entries correspond to components of the mathematical vector in the optimization model.



In [ ]:
import numpy as np

x = np.array([1,3,4,1])
x_range = np.arange(1,10)

print(x)
print(x_range)

[1 3 4 1]
[1 2 3 4 5 6 7 8 9]


Vectors are essential objects especially in linear algebra related operations. This would entitle to a separate discussion altogether. However, to appreciate the simplicity of mathematical modeling, let's provide some examples.

For a linear transformation $A$ acting on a set of variables $\dot{x}$ producing some vector $y$ we can let $A$ be a matrix:

$$A = \frac{1}{3}\begin{pmatrix}-1&1 \\ 0&2\end{pmatrix}$$

and $\dot{x}$ and $y$ are one-dimensional vectors:

$$\dot{x} = \begin{pmatrix}3\\4\end{pmatrix}$$

We can then express this as:
$$A\dot{x} = y$$

In [ ]:
A = np.array([
    [-1, 1],
    [0, 2]
])
x_dot = np.array([[3,4]]).T

y = A @ x_dot
print(y)

[[1]
 [8]]


### 2.5 Set Builder Notation Algorithms

In mathematics, **set-builder notation** describes a set by specifying how its elements are generated and which properties they satisfy, for example
$$
S = \{\, x \in \mathbb{R} \mid x \ge 0,\; f(x) \le 1 \,\},
$$
which reads “the set of all real $x$ such that $x \ge 0$ and $f(x) \le 1$.” In Python, list comprehensions play a similar role for building lists: they construct new collections from existing domains by combining a generation rule with optional filtering conditions.  

The basic syntax of a list comprehension is
$$
\texttt{[expression for item in iterable if condition]},
$$
where the expression corresponds to the mapping part, and the condition corresponds to the predicate in set-builder notation. For example, the mathematical set
$$
\{\, x^2 \mid x \in \{0,1,\dots,9\},\; x \text{ even} \,\}
$$
can be mirrored in Python as

In [ ]:
[x**2 for x in range(10) if x % 2 == 0]

[0, 4, 16, 36, 64]

### **Activity 2**: Operating with Collections

A. Encode the following set builder notations as Python functions. The return values of your functions should be a `list`.
1. $A = \{2^n | n \in \mathbb{Z}^+,n\leq10\}$
2. $B = \{\frac{dy}{dx}x+c_0 | x \in \mathbb{Z}^-, n \leq 12\}$
2. $C = \{x^2 + y^2 | (x,y) \in \mathbb{R}^+, x \leq \pi ,y \leq \pi\}, |C| =20$

B. Create a function to generate a list which it has the following values:
$$F = \{0,1,1,2,3,5,8, ... , 80\}$$

*Hint*: $f_i \in F, f_i  =
\begin{cases}
  0      & i = 0, \\
  1      & i = 1, \\
  f_{i-1} + f_{i-2}      & \text{otherwise}
\end{cases}$

## 3. Iterative Operations

In optimization, many algorithms are iterative: they repeatedly apply a function, accumulate quantities like sums or products, and update candidate solutions. Python's 'for' loops provide a direct way to translate these iterative mathematical operations into code.



### 3.1 Running functions in a loop

Given a function $f(x)$, a common task is to evaluate it at many candidate points: $x_0, x_1, ... , x_{n-1}$. In Python, this becomes:

In [ ]:
def objective(x: float) -> float:
    return (x - 3)**2 + 1

candidates = [-2, -1, 0, 1, 2, 3, 4]
values = []

for x in candidates:
    fx = objective(x)
    values.append(fx)
    print(f'x = {x}\tf(x) = {fx}')


x = -2	f(x) = 26
x = -1	f(x) = 17
x = 0	f(x) = 10
x = 1	f(x) = 5
x = 2	f(x) = 2
x = 3	f(x) = 1
x = 4	f(x) = 2


Here, the loop implements "for each $x$ in the candidate set, compute $f(x)$" which corresponds directly to evaluating the objective over a finite subset of the feasible region. Similar patterns appear when repeatedly calling constraint functions, gradient approximations, or update rules in iterative optimization methods.

### 3.2 Translating summation ($\Sigma$) into loops
The looping or iteration tecnique in programming greatly helps in operating with collections such as sets, ranges, and vectors. Most of these operations are called Big Operators, which include summation and product over a set. In the next few cells, we will convert big operations as Python

The mathematical summation is expressed as:

The mathematical summation can be done over elements of a collection or by specifying a progression of a range. The expression:
$$\sum_{n\in N} n$$
Takes the sum all elements $n$ in a set $N$. While the expression:
$$\sum^N_{i=2}n_i$$
Takes the sum of elements $n$ in the set $N$, wherein we start with the second element of the collection. In Python, we can freely express these nuances in the mathematical expression.

In [ ]:
N = np.arange(1,100)

## considering the for all form
total = 0
for n in N:
  total += n

print(total)

4950


In [ ]:
## using the indexed form
start = 2
stop = len(N)

total = 0
for i in range(start, stop):
  total += N[i]

print(total)

4947


However, packages such as math and numpy simplifies this expression as an inline expression making modeling more abstract:


In [ ]:
np.sum(N)

np.int64(4950)

Similarly, for the product operator $\Pi$, we can also apply the same mathematical routine. For example:

$$\prod_{s\in S}p(s)$$

Suppose that $S$ is a set of scenarios and $p$ is a probability function that determines the likelihood of $s$.

In [ ]:
S = np.array([300,200,123,542,867,113])
max_count = 1000

def prob(s):
  return s/max_count

P = 1
for s in S:
  P *= prob(s)
P

np.float64(0.00039188008116000005)

In [ ]:
np.prod(S/max_count)

np.float64(0.00039188008116000005)

Notice that we did not use list comprehension in the previous function. This is one advantage in working with vectors. Numpy enables "vectorization" and broadcasting for vectors.

### **Activity 3**: Iterations in Algorithms

Create respective functions that correspond to the following mathematical expressions:
1. $$\sqrt{\frac{1}{N}\sum^N_{i=0}(\dot{y_i} - \hat{y_i})^2}$$

In [ ]:
np.random.seed(333)
y_dot = np.random.randint(1,3,10)
y_hat = np.random.randint(1,3,10)
## DO NOT EDIT ABOVE THIS LINE

### START CODING HERE


2. Given the matrix $G$:
$$G = \begin{pmatrix}1&0.3&0.4&0.2&0.12 \\ 0&3&0.11&0.32&0.01 \\ 50&0&2&0.65&0.01 \\ 0&0&0&1&0.22\ \\ 124&0&100&0&3\end{pmatrix}$$

Compute for the sum of the upper triangular given as:
$$\sum^N_{i=0}\sum^M_{j\leq i}G_{ij}$$

In [ ]:
### START CODING HERE


3. Solve for $x$ corresponding to the global minima of the function $f(x) = \frac{1}{3}x^2 + 2x -1.12$. Use a `for` loop to iterate over the range of $x$. We note that $x \in \mathbb{R}$ in this problem.

In [ ]:
### START CODING HERE




---

